# IEMOCAP

#### * Identifier:
USC_IEMOCAP
	
#### * Modalities:
Audio, video, Motion Capture
	
#### * Emotional content:
angry, happy, sad, neutral, frustrated, excited, fearful, disgusted, excited, other	

#### * Emotion elicitation methods:
scripts and improvisation	

#### * Size:
10 subjects, ~12 hours of audiovisual data
	
#### * Nature of material:
dyadic interactions	

#### * Language:
English

#### * Reference:
Busso, Carlos et al.  "IEMOCAP: Interactive emotional dyadic motion capture database."  Journal of Language Resources and Evaluation.  Volume 42, Number 4.  pp. 335-359.  2008

# Label  
% [START_TIME - END_TIME] TURN_NAME EMOTION [V, A, D]

[6.2901 - 8.2357]	Ses01F_impro01_F000	neu	[2.5000, 2.5000, 2.5000]  
C-E2:	Neutral;	()  
C-E3:	Neutral;	()  
C-E4:	Neutral;	()  
C-F1:	Neutral;	(curious)  
A-E3:	val 3; act 2; dom  2;	()  
A-E4:	val 2; act 3; dom  3;	(mildly aggravated but staying polite, attitude)  
A-F1:	val 3; act 2; dom  1;	()  
  
[10.0100 - 11.3925]	Ses01F_impro01_F001	neu	[2.5000, 2.5000, 2.5000]  
C-E2:	Neutral;	()  
C-E3:	Neutral;	()  
C-E4:	Neutral;	()  
C-F1:	Neutral; Anger;	()  
A-E3:	val 3; act 2; dom  2;	()  
A-E4:	val 2; act 3; dom  3;	(guarded, tense, ready)  
A-F1:	val 2; act 3; dom  2;	()  
  
[14.8872 - 18.0175]	Ses01F_impro01_F002	neu	[2.5000, 2.5000, 2.5000]  
C-E2:	Neutral;	()  
C-E3:	Surprise;	()  
C-E4:	Neutral;	()  
C-F1:	Neutral; Anger;	()  
A-E3:	val 3; act 2; dom  2;	()  
A-E4:	val 2; act 3; dom  3;	(superior, indifferent, menacing)  
A-F1:	val 3; act 3; dom  3;	()  
  
[19.2900 - 20.7875]	Ses01F_impro01_F003	xxx	[2.5000, 3.0000, 3.0000]  
C-E2:	Neutral;	()  
C-E3:	Frustration;	()  
C-E4:	Anger;	()  
C-F1:	Frustration; Anger;	()  
A-E3:	val 3; act 3; dom  3;	()  
A-E4:	val 2; act 3; dom  3;	(accusing)  
A-F1:	val 2; act 3; dom  3;	()

In [ ]:
import os, re
import librosa
from tqdm.notebook import tqdm
import numpy as np
import torch
from utils import *
import random
random.seed(0)

data_path = "./Dataset/IEMOCAP"

if not os.path.exists(f"{data_path}/melspectrogram"):
    os.mkdir(f"{data_path}/melspectrogram")
    os.mkdir(f"{data_path}/melspectrogram/train")
    os.mkdir(f"{data_path}/melspectrogram/validation")
    os.mkdir(f"{data_path}/melspectrogram/test")

Sessions = [1,2,3,4,5]
labels = ['ang', 'hap', 'sad', 'neu', 'fru', 'exc', 'sur', 'fea', 'dis', 'oth', 'xxx']
label2id = dict([ (l,i) for i, l in enumerate(labels)])
label_count = [0, 0, 0, 0, 0, 0, 0]
data_list=[]
for s in Sessions:
    root_dir = f'./Dataset/IEMOCAP/Session{s}/dialog/EmoEvaluation'
    dialogues = sorted([f for f in os.listdir(root_dir) if (f.endswith('.txt')) and not f.startswith('.')])
    for dialogue in dialogues:
        with open(os.path.join(root_dir, dialogue), 'r') as f:
            lines = f.read().splitlines()
            for line in lines:
                if len(line)>0 and line[0]=='[' and line[-1]==']':
                    basename, label = line.split()[3], line.split()[4]
                    session, part, speaker = basename[4], basename[:-5], basename[:6]
                    assert label in labels
                    assert speaker in ['Ses01M', 'Ses01F', 'Ses02M', 'Ses02F', 'Ses03M', 'Ses03F', 'Ses04M', 'Ses04F', 'Ses05M', 'Ses05F']
                    
                    if label in ['ang', 'hap', 'sad', 'neu', 'fru', 'exc', 'sur']:
                        label_count[ label2id[label] ] += 1
                        data_list.append((f'{data_path}/Session{session}/sentences/wav/{part}/{basename}.wav', label))
                        
random.shuffle(data_list)

##### Train Set #####
for wav_path, label in tqdm(data_list[2000:]):
    y, sr = librosa.load(wav_path, sr=16000)
    wav_name = f"{label}_{wav_path.split('/')[-1][:-4]}"
    mel = mel_spectrogram_torch(torch.from_numpy(y).unsqueeze(0), 1024, 80, 16000, 256, 1024, 0.0, 8000.0, center=False)
    np.save(f"{data_path}/melspectrogram/train/{wav_name}.npy", mel[0].numpy())
    
##### Validation Set #####
for wav_path, label in tqdm(data_list[1000:2000]):
    y, sr = librosa.load(wav_path, sr=16000)
    wav_name = f"{label}_{wav_path.split('/')[-1][:-4]}"
    mel = mel_spectrogram_torch(torch.from_numpy(y).unsqueeze(0), 1024, 80, 16000, 256, 1024, 0.0, 8000.0, center=False)
    np.save(f"{data_path}/melspectrogram/validation/{wav_name}.npy", mel[0].numpy())
    
##### Test Set #####
for wav_path, label in tqdm(data_list[:1000]):
    y, sr = librosa.load(wav_path, sr=16000)
    wav_name = f"{label}_{wav_path.split('/')[-1][:-4]}"
    mel = mel_spectrogram_torch(torch.from_numpy(y).unsqueeze(0), 1024, 80, 16000, 256, 1024, 0.0, 8000.0, center=False)
    np.save(f"{data_path}/melspectrogram/test/{wav_name}.npy", mel[0].numpy())